In [ ]:
import pandas as pd
import numpy as np

In [ ]:
lv_df = pd.read_csv('/Users/ethanweber/Downloads/LV_df.csv')

In [ ]:
def fix_categories(x):
    x = str(x)
    x = x.replace('[', '').replace(']', '').replace(',', '').split("u'")
    x = map(lambda y: y.replace("'", ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        return filter(None, x)
    else:
        return x

lv_df['categories'] = map(fix_categories, lv_df['categories'])

In [ ]:
def get_big_cat(z):
    catlist = ['Active Life', 'Arts & Entertainment', 'Automotive', 'Beauty & Spas', 'Education', 'Event Planning & Services', 'Financial Services', 'Food', 'Health & Medical', 'Home Services', 'Hotels & Travel', 'Local Flavor', 'Local Services',  'Nightlife', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations', 'Restaurants', 'Shopping']
    return filter(lambda x: x in catlist, z)
lv_df['big_cat'] = map(get_big_cat, lv_df['categories'])

In [ ]:
def get_little_cat(z):
    catlist = ['Active Life', 'Arts & Entertainment', 'Automotive', 'Beauty & Spas', 'Education', 'Event Planning & Services', 'Financial Services', 'Food', 'Health & Medical', 'Home Services', 'Hotels & Travel', 'Local Flavor', 'Local Services',  'Nightlife', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations', 'Restaurants', 'Shopping']
    return filter(lambda x: (x in catlist) == False, z)
lv_df['categories'] = map(get_little_cat, lv_df['categories'])


In [ ]:
def get_excluder(z):
    rmlist = ['Automotive', 'Education', 'Financial Services', 'Mass Media', 'Home Services', 'Local Services', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations']
    for i in rmlist:
        if i in z:
            return 1
    return 0

lv_df['rm'] = map(get_excluder, lv_df['big_cat'])

In [ ]:
lv_df = lv_df[lv_df['rm'] == 0]


In [ ]:
lv_df[1:5]

In [ ]:
del lv_df2['index']
del lv_df2['Unnamed: 0']

In [ ]:
lv_df2

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned

#lv_df['text'] = lv_df['text'].apply(lambda x: cleaning_text(x))

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import gensim
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')

lmtizer=nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer() # Create our stemmer 
stem_lem = lambda x: lmtizer.lemmatize(stemmer.stem(x))

In [ ]:
def split_stem_lem(x):
    try:
        x = map(lambda y: stem_lem(y), cleaning_text(x).split())
        return x
    except:
        print 'Fuck'
        return None
    


In [ ]:

lv_df = pd.read_csv('lv_df_final.csv', sep='|', encoding='utf-8')


In [ ]:
lv_df['text2'] = map(split_stem_lem, lv_df['text'])


In [1]:
def fix_text2(x):
    x = str(x)
    x = x.replace('[', '').replace(']', '').replace(',', '').split(" ")
    x = map(lambda y: y.replace("'", ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        return filter(None, x)
    else:
        return x

In [2]:
import pandas as pd
import numpy as np

print 'going...'
lv_df = pd.read_csv('lv_df_final.csv', sep='|', encoding='utf-8')
print 'loaded'
lv_df['text2'] = map(fix_text2, lv_df['text2'])
print 'done'

going...


/Users/ethanweber/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,4,6,17,18,32,34,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


loaded
done


In [4]:
lv_df2 = lv_df
del lv_df2['text']

KeyError: 'text'

In [5]:
del lv_df2['Unnamed: 0']

In [9]:
flattened = [val for sublist in list(lv_df2['text2']) for val in sublist]

In [16]:
tokens = flattened

In [26]:
corpus = list(lv_df2['text2'])

In [45]:
clipped_corpus = gensim.utils.ClippedCorpus(corpus, max_docs=None) 
id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_wordids.txt')
lsi = gensim.models.lsimodel.LsiModel(corpus=clipped_corpus, id2word=id2word, num_topics=20)

done


In [47]:
print clipped_corpus

In [ ]:
def recover_cat_list(x):
    x = str(x).replace('[', '').replace(']', '').split(',')
    x = map(lambda y: y.replace("'", ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        return filter(None, x)
    else:
        return x

In [ ]:
lv_df['categories'] = map(recover_cat_list, lv_df['categories'])

In [ ]:
lv_df['big_cat'] = map(recover_cat_list, lv_df['big_cat'])

In [13]:
import logging
import os
import sys
import re
import tarfile
import itertools


import nltk
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures


import gensim
from gensim.parsing.preprocessing import STOPWORDS

def flatten(x):
    return sum(list(x), [])

def tokenize(text):
    return [token for token in gensim.utils.simple_preprocess(text) if token not in STOPWORDS]

In [ ]:
col_list = list(lv_df2['text2'])
#word_list = sum(, [])

In [19]:
vector = next(iter(tokens))

'person'

In [ ]:
#lv_df['categories'] = map(fix_categories, lv_df['categories'])


In [ ]:
#catlist = ['Active Life', 'Arts & Entertainment', 'Automotive', 'Beauty & Spas', 'Education', 'Event Planning & Services', 'Financial Services', 'Food', 'Home Services', 'Health & Medical', 'Hotels & Travel', 'Local Flavor', 'Local Services', 'Mass Media', 'Nightlife', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations', 'Restaurants', 'Shopping']
#rmlist = ['Automotive', 'Education', 'Financial Services',  'Health & Medical', 'Mass Media', 'Home Services', 'Local Services', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations']
#new_list = ['Active Life', 'Arts & Entertainment', 'Beauty & Spas', 'Event Planning & Services', 'Food', 'Hotels & Travel', 'Local Flavor', 'Nightlife', 'Restaurants', 'Shopping']
#print len(sorted((rmlist) + (new_list)))
#print len(sorted(catlist))

In [ ]:
#lv_df.to_pickle('lv_df.pkl')  # where to save it, usually as a .pkl

#lv_df = pd.read_pickle('lv_df.pkl')


In [ ]:
lv_df2 = lv_df
del lv_df2['text']


In [ ]:
active_df = lv_df2[map(lambda x: 'Active Life' in x, lv_df2['big_cat'])]
arts_df = lv_df2[map(lambda x: 'Arts & Entertainment' in x, lv_df2['big_cat'])]
beauty_df = lv_df2[map(lambda x: 'Beauty & Spas' in x, lv_df2['big_cat'])]
events_df = lv_df2[map(lambda x: 'Event Planning & Services' in x, lv_df2['big_cat'])]
food_df = lv_df2[map(lambda x: 'Food' in x, lv_df2['big_cat'])]
hotels_df = lv_df2[map(lambda x: 'Hotels & Travel' in x, lv_df2['big_cat'])]
local_df = lv_df2[map(lambda x: 'Local Flavor' in x, lv_df2['big_cat'])]
nightlife_df = lv_df2[map(lambda x: 'Nightlife' in x, lv_df2['big_cat'])]
restaurant_df = lv_df2[map(lambda x: 'Restaurants' in x, lv_df2['big_cat'])]
shops_df = lv_df2[map(lambda x: 'Shopping' in x, lv_df2['big_cat'])]

In [ ]:
active_df.to_csv('active_df.csv', sep='|', encoding='utf-8')
arts_df.to_csv('arts_df.csv', sep='|', encoding='utf-8')
beauty_df.to_csv('beauty_df.csv', sep='|', encoding='utf-8')
events_df.to_csv('events_df.csv', sep='|', encoding='utf-8')
food_df.to_csv('food_df.csv', sep='|', encoding='utf-8')
hotels_df.to_csv('hotels_df.csv', sep='|', encoding='utf-8')
local_df.to_csv('local_df.csv', sep='|', encoding='utf-8')
nightlife_df.to_csv('nightlife_df.csv', sep='|', encoding='utf-8')
restaurant_df.to_csv('restaurant_df.csv', sep='|', encoding='utf-8')
shops_df.to_csv('shops_df.csv', sep='|', encoding='utf-8')

In [ ]:
active_df.to_pickle('./pickles/active_df.pkl')
arts_df.to_pickle('./pickles/arts_df.pkl')
beauty_df.to_pickle('./pickles/beauty_df.pkl')
events_df.to_pickle('./pickles/events_df.pkl')
food_df.to_pickle('./pickles/food_df.pkl')
hotels_df.to_pickle('./pickles/hotels_df.pkl')
local_df.to_pickle('./pickles/local_df.pkl')
nightlife_df.to_pickle('./pickles/nightlife_df.pkl')
restaurant_df.to_pickle('./pickles/restaurant_df.pkl')
shops_df.to_pickle('./pickles/shops_df.pkl')

In [ ]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [ ]:
n_features = 1000
n_topics = 10
n_top_words = 20

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words='english')

In [ ]:
def iter_wiki(dump_file):
    """Yield each article from the Wikipedia dump, as a `(title, tokens)` 2-tuple."""
    ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for title, text, pageid in _extract_pages(smart_open(dump_file)):
        text = filter_wiki(text)
        tokens = tokenize(text)
        if len(tokens) < 50 or any(title.startswith(ns + ':') for ns in ignore_namespaces):
            continue  # ignore short articles and various meta-articles
        yield title, tokens

In [ ]:

tf = tf_vectorizer.fit_transform(text_train)
TF = TfidfTransformer()
tf_idf = TF.fit_transform(tf)
tf_idf = tf_idf.toarray() #convert sparse matrix to array
#tf_idf.toarray()[:100, :100]

In [ ]:
1+1

In [ ]:
#LV_samplei.categories = map(lambda x: fix_categories(str(x)) , LV_samplei.categories)

In [ ]:
import gensim
from gensim import corpora
df_list = [active_df, arts_df, beauty_df, events_df, food_df, hotels_df, local_df, nightlife_df, restaurant_df, shops_df]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel

In [ ]:
active_dict = corpora.Dictionary(active_df.categories)
arts_dict = corpora.Dictionary(arts_df.categories)
beauty_dict = corpora.Dictionary(beauty_df.categories)
events_dict = corpora.Dictionary(events_df.categories)
food_dict = corpora.Dictionary(food_df.categories)
hotels_dict = corpora.Dictionary(hotels_df.categories)
local_dict = corpora.Dictionary(local_df.categories)
nightlife_dict = corpora.Dictionary(nightlife_df.categories)
restaurant_dict = corpora.Dictionary(restaurant_df.categories)
shops_dict = corpora.Dictionary(shops_df.categories)

In [ ]:
doc_term_active = [active_dict.doc2bow(x) for x in active_df.categories]
doc_term_arts = [arts_dict.doc2bow(x) for x in arts_df.categories]
doc_term_beauty = [beauty_dict.doc2bow(x) for x in beauty_df.categories]
doc_term_events = [events_dict.doc2bow(x) for x in events_df.categories]
doc_term_food = [food_dict.doc2bow(x) for x in food_df.categories]
doc_term_hotels = [hotels_dict.doc2bow(x) for x in hotels_df.categories]
doc_term_local = [local_dict.doc2bow(x) for x in local_df.categories]
doc_term_nightlife = [nightlife_dict.doc2bow(x) for x in nightlife_df.categories]
doc_term_restaurant = [restaurant_dict.doc2bow(x) for x in restaurant_df.categories]
doc_term_shops = [shops_dict.doc2bow(x) for x in shops_df.categories]

In [ ]:
lda_active = Lda(doc_term_active, num_topics=3, id2word = active_dict, passes=3)
print 'active done'
lda_arts = Lda(doc_term_arts, num_topics=5, id2word = arts_dict, passes=3)
print 'arts done'
lda_beauty = Lda(doc_term_beauty, num_topics=3, id2word = beauty_dict, passes=3)
print 'beauty done'
lda_events = Lda(doc_term_events, num_topics=7, id2word = events_dict, passes=3)
print 'events done'
lda_food = Lda(doc_term_food, num_topics=10, id2word = food_dict, passes=3)
print 'food done'
lda_hotels = Lda(doc_term_hotels, num_topics=6, id2word = hotels_dict, passes=3)
print 'hotels done'
lda_local = Lda(doc_term_local, num_topics=3, id2word = local_dict, passes=3)
print 'local done'
lda_nightlife = Lda(doc_term_nightlife, num_topics=3, id2word = nightlife_dict, passes=3)
print 'nightlife done'
lda_restaurant = Lda(doc_term_restaurant, num_topics=45, id2word = restaurant_dict, passes=3)
print 'restaurants done'
lda_shops = Lda(doc_term_shops, num_topics=20, id2word = shops_dict, passes=3)
print 'shops done'

In [ ]:
lda_active.save('./lda_pickles/lda_active.pkl')
lda_arts.save('./lda_pickles/lda_arts.pkl')
lda_beauty.save('./lda_pickles/lda_beauty.pkl')
lda_events.save('./lda_pickles/lda_events.pkl')
lda_food.save('./lda_pickles/lda_food.pkl')
lda_hotels.save('./lda_pickles/lda_hotels.pkl')
lda_local.save('./lda_pickles/lda_local.pkl')
lda_nightlife.save('./lda_pickles/lda_nightlife.pkl')
lda_restaurant.save('./lda_pickles/lda_restaurant.pkl')
lda_shops.save('./lda_pickles/lda_shops.pkl')

In [ ]:
lda_list = [lda_active, lda_arts, lda_beauty, lda_events, lda_food, lda_hotels, lda_local, lda_nightlife, lda_restaurant, lda_shops] 

In [ ]:
for i in lda_list:
    print i
    print i.print_topics(num_topics=3, num_words=3)[0]

In [ ]:
# As I want to avoid dealing with a huge sparse matrix, I will only select the top most reviewed books for our example 

most_reviewed_biz = pd.DataFrame({'count' : LV_test.groupby(["name_x"]).size()})\
                                    .reset_index().sort(['count'],ascending = False)

most_reviewed_biz.head(20) 

In [ ]:
# Let's create a function that collect the reviews of our common reviewers
def get_biz_reviews(biz, common_reviewers):
    mask = (LV_test.user_id.isin(common_reviewers)) & (LV_test.name_x==biz)
    reviews = LV_test[mask].sort('user_id')
    reviews = reviews[reviews.user_id.duplicated()==False]
    return reviews

In [ ]:
# Importing the scipy library to measure the pearson correlation coefficient
from scipy.stats.stats import pearsonr
def calculate_correlation(biz1, biz2):
    # We start by finding the common reviewers
    biz_1_reviewers = LV_test[LV_test.name_x == biz1].user_id
    biz_2_reviewers = LV_test[LV_test.name_x == biz2].user_id
    common_reviewers = set(biz_1_reviewers).intersection(biz_2_reviewers)

    # Then we look for the reviews given by common reviewers
    biz_1_reviews = get_biz_reviews(biz1, common_reviewers)
    biz_2_reviews = get_biz_reviews(biz2, common_reviewers)
    
    # Calculate the Pearson Correlation Score
    return pearsonr(biz_1_reviews.stars_x, biz_2_reviews.stars_x)[0]

In [ ]:
# Getting the list of the most reviewed biz

top_biz = []

for i in most_reviewed_biz.name_x[0:13]:
    top_biz.append(i)

    
# calculate the correlation for our top biz
correlation_coefficient = []

for biz1 in top_biz:
    print "Calculating the correlations for:", biz1
    for biz2 in top_biz:
        if biz1 != biz2:
            row = [biz1, biz2] + [calculate_correlation(biz1, biz2)]
            correlation_coefficient.append(row)

In [ ]:
# Let's look at what the table of correlation looks like
cols = ["biz_1", "biz_2", "Correlation"]
correlation_coefficient = pd.DataFrame(correlation_coefficient, columns=cols).sort('Correlation')
correlation_coefficient.head(10)

In [ ]:
def calc_correlation(corr, biz1, biz2):
    mask = (corr.biz_1==biz1) & (corr.biz_2==biz2)
    row = corr[mask]
    corr = row
    return corr.sum(axis=1).tolist()[0]

In [ ]:
calc_correlation(correlation_coefficient,"McDonald's", "Smashburger")